### Imports

In [1]:
import shutil
import csv
import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from packages.metagenomics import sampling2, encoding2
from sklearn.linear_model import LogisticRegression

from packages.LogisticRegression.MulticlassLogisticRegression import MulticlassLogisticRegression,MulticlassLogisticRegression2


In [2]:
def append_results_to_file(filename, fields=None, rows=None):
    
    with open(filename, 'a') as f:

        write = csv.writer(f)

        if fields:
            write.writerow(fields)

        if rows:
            write.writerows(rows)

In [3]:
def run_mlr_classification_recall(X_train, X_test, y_train, y_test, eta, epsilon):
    """
    Score is species level recall.
    """
    mlr = MulticlassLogisticRegression2(eta=eta, epsilon=epsilon)
    mlr.fit(X_train,y_train)
    y_pred = mlr.predict(X_test)
    score = recall_score(y_test, y_pred, average='weighted')
    return score
    

In [4]:
def build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed):
    # delete output directory if it previously exists
    try:
        shutil.rmtree(output_dir)
    except FileNotFoundError:
        print('Existing directory was not found. Process will generate a directory.')

    # build fragments
    sampling2.generate_fragment_data(seq_file, taxid_file, output_dir, sample_length, coverage, seed)

In [5]:
def encode_fragments(output_dir, pattern, k, seed):
    
    # encode data
    fragments = sampling2.read_fragments(output_dir, pattern)
    X_enc, y = encoding2.encode_fragment_dataset(fragments,k)
    le = preprocessing.LabelEncoder()
    y_enc = le.fit_transform(y)
    
    # calculate number of classes
    n_classes = len(np.unique(y_enc))
#     print('n_classes:',n_classes)
    n_classes_train = 0
    n_classes_test = 0
    while n_classes_train < n_classes or n_classes_test < n_classes:

        # split data into test and training
        X_train, X_test, y_train, y_test = train_test_split(X_enc.toarray(), y_enc, test_size=0.33)
        n_classes_train = len(np.unique(y_train))
        n_classes_test = len(np.unique(y_test))
#         print('train:',len(y_train))
#         print('test:', len(y_test))
    
    print(X_enc.shape)
    
    return X_train, X_test, y_train, y_test

In [6]:
def calc_number_combinations(list_sample_length,list_coverage,list_k,list_eta,list_epsilon):
    n = len(list_sample_length) * len(list_coverage) * len(list_k) *len(list_eta) * len(list_epsilon)
    return n

In [7]:
def grid_search_multiclass_mlr(seq_file, 
                              taxid_file, 
                              output_dir, 
                              pattern, 
                              list_sample_length, 
                              list_coverage, 
                              list_k,
                              list_eta,
                              list_epsilon, 
                              seed,
                              grid_search_file,
                              fields,
                              experiment,
                              score_type):
    
    # set up grid search results file
    append_results_to_file(grid_search_file, fields)
    
    # calculate number of combinations
    n_combinations = calc_number_combinations(list_sample_length,list_coverage,list_k,list_eta,list_epsilon)
    
    # process combinations
    count = 0
    for sample_length in list_sample_length:
        for coverage in list_coverage:
            
            # fragment combination
            build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed)
            for k in list_k:
                
                # kmer combination
                X_train, X_test, y_train, y_test = encode_fragments(output_dir, pattern,k,seed)
                for eta in list_eta:
                    for epsilon in list_epsilon:
                        
                        # random forest combination
                        score = run_mlr_classification_recall(X_train, X_test, y_train,y_test, eta, epsilon)
                        count += 1
                        
                        # output results to file
                        row = [experiment, 'multiclass', 'Logistic Regression', X_train.shape, sample_length, coverage, k, eta, epsilon, score, score_type]
                        append_results_to_file(grid_search_file, row)
                        
                print('Percent complete: {}'.format(count / n_combinations * 100)) # display progress
    

# Run Set 1 - MLR Toy
2000 lengths dataset

### Run 4.01
Stopped early due to runs taking a long time.

In [8]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.01'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [0.2,1,10]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 4.02

In [9]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.02'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [200]
# list_coverage = [0.3,1,10]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 4.03

In [10]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.03'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [400]
# list_coverage = [0.5,1,10]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 4.04

In [11]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.04'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [100]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


(8141, 400)
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 14.285714285714285
(8141, 800)
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 28.57142857142857
(8141, 6399)
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. 

### Run 4.05

In [18]:
# %%time
##40 min

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.05'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [200]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


(16282, 400)
n_classifiers 5
training classifier 0
training classifier 1


/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:283: RuntimeWarning: overflow encountered in exp
  :param X: L x n matrix, where L is the number of samples and n is the number of features
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:235: RuntimeWarning: overflow encountered in exp
  :param X: L x n matrix, where L is the number of samples and n is the number of features
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:238: RuntimeWarning: invalid value encountered in true_divide
  """


training classifier 2
training classifier 3
training classifier 4
Percent complete: 14.285714285714285
(16282, 800)
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 28.57142857142857
(16282, 6400)
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 42.857142857142854
(16282, 43939)
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMI

# Run Set 2 - sklearn
Compare with sklearn implementation of MLR to see if performance is similar.

In [12]:
def calc_number_combinations2(list_sample_length,list_coverage,list_k,list_multiclass,list_classweight):
    n = len(list_sample_length) * len(list_coverage) * len(list_k) *len(list_multiclass) * len(list_classweight)
    return n

In [13]:
def encode_fragments2(output_dir, pattern, k, seed):
    """
    Does not convert sparse matrix to numpy matrix first.
    """
    
    # encode data
    fragments = sampling2.read_fragments(output_dir, pattern)
    X_enc, y = encoding2.encode_fragment_dataset(fragments,k)
    le = preprocessing.LabelEncoder()
    y_enc = le.fit_transform(y)
    
    # calculate number of classes
    n_classes = len(np.unique(y_enc))
#     print('n_classes:',n_classes)
    n_classes_train = 0
    n_classes_test = 0
    while n_classes_train < n_classes or n_classes_test < n_classes:

        # split data into test and training
        X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=0.33)
        n_classes_train = len(np.unique(y_train))
        n_classes_test = len(np.unique(y_test))
#         print('train:',y_train)
#         print('test:', y_test)
    
    print(X_enc.shape)
    
    return X_train, X_test, y_train, y_test

In [14]:
def run_lr_classification_recall(X_train, X_test, y_train, y_test, multiclass, classweight, seed ):
    """
    Score is species level recall. Uses sklearn version of logistic regression.
    """
    lr = LogisticRegression(random_state=seed, multi_class=multiclass, class_weight=classweight )
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)
    score = recall_score(y_test, y_pred, average='weighted')
    return score
    

In [15]:
def grid_search_multiclass_lr(seq_file, 
                              taxid_file, 
                              output_dir, 
                              pattern, 
                              list_sample_length, 
                              list_coverage, 
                              list_k,
                              list_multiclass,
                              list_classweight,
                              seed,
                              grid_search_file,
                              fields,
                              experiment,
                              score_type):
    
    # set up grid search results file
    append_results_to_file(grid_search_file, fields)
    
    # calculate number of combinations
    n_combinations = calc_number_combinations2(list_sample_length,list_coverage,list_k, list_multiclass, list_classweight)
    
    # process combinations
    count = 0
    for sample_length in list_sample_length:
        for coverage in list_coverage:
            
            # fragment combination
            build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed)
            for k in list_k:
                
                # kmer combination
                X_train, X_test, y_train, y_test = encode_fragments2(output_dir, pattern,k,seed)
                
                
                for multiclass in list_multiclass:
                    for classweight in list_classweight:

                        
                        # random forest combination
                        score = run_lr_classification_recall(X_train, X_test, y_train, y_test, multiclass, classweight, seed)
                        count += 1

                        # output results to file
                        row = [experiment, 'multiclass', 'Logistic Regression (sklearn)', X_train.shape, sample_length, coverage, k, multiclass, classweight, score, score_type]
                        append_results_to_file(grid_search_file, row)

                print('Percent complete: {}'.format(count / n_combinations * 100)) # display progress


### Run 5.01

In [16]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','multiclass', 'class_weight', 'score','score type']
# experiment = '5.01'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [1]
# list_multiclass = ['auto']
# list_classweight= [None]


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 5.02

In [17]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','multiclass', 'class_weight', 'score','score type']
# experiment = '5.02'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [2,4,6,8,10,12]
# list_multiclass = ['auto']
# list_classweight= [None]


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)
